# Получение вопроса, поиск нужных чанков, генерация ответа

### загрузка модели из data

In [7]:
# создаём путь до места хранения
from pathlib import Path

# Получаем текущий рабочий каталог
current_dir = Path.cwd()

# Создаем путь к папке processed_data, которая находится на уровне выше текущего каталога
processed_data_path = current_dir.parent / 'data' / 'processed_data' / 'union.parquet'


In [8]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings

vectorstore = SKLearnVectorStore(
    embedding=OllamaEmbeddings(model="llama3.1"), 
    persist_path=processed_data_path, 
    serializer="parquet"
)

retriever = vectorstore.as_retriever(k=4)


### Вопрос, поиск ответа

In [9]:
question = "Что такое базы данных и зачем они нужны"

documents = retriever.invoke(question)
# Extract content from retrieved documents
doc_texts = "\\n".join([doc.page_content for doc in documents])

In [11]:
print(len(doc_texts))
doc_texts

3114


'Контрольные вопросы и задания . . . . . . . . . . . . . . . . . . . . . . . . . . . 29\nГлава 3. Основные операции с таблицами 31\nКонтрольные вопросы и задания . . . . . . . . . . . . . . . . . . . . . . . . . . . 48\nГлава 4. Типы данных СУБД PostgreSQL 51\n4.1. Числовые типы . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 51\n4.2. Символьные (строковые) типы . . . . . . . . . . . . . . . . . . . . . . . . . 54\n4.3. Типы «дата/время» . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 56\n4.4. Логический тип . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 63\n4.5. Массивы . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 64\n4.6. Типы JSON . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 68\nКонтрольные вопросы и задания . . . . . . . . . . . . . . . . . . . . . . . . . . . 73\nГлава 5. Основы языка определения данных 95\\nОглавление\nПредисловие автора 5\nВведение 7\nГлава 1. Вве

### Развёртывание модели

In [12]:
from langchain.prompts import PromptTemplate

# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""Вы помощник в вопросах-ответах.
    Для ответа на вопрос используйте следующие документы если они полезны.
    Если вы не знаете ответа, поробуйте ответить самостоятельно или просто скажите, что не знаете.
    Вопрос {question}
    Документы: {documents}
    Отвечать:
    """,
    input_variables=["question", "documents"],
)

In [25]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model='llama3.1', temperature=0)


C:\Users\dimka\AppData\Local\Temp\ipykernel_7576\1894568862.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model='llama3.1', temperature=0)


In [27]:
# Create a chain combining the prompt template and LLM
rag_chain = prompt | llm | StrOutputParser()

### Пердаём данные модели и получаем ответ

In [28]:
answer = rag_chain.invoke({"question": question, "documents": doc_texts})

In [30]:
# Вопрос
question

'Что такое базы данных и зачем они нужны'

In [31]:
# Что выдал поиск по чанкам (Основа rag)
doc_texts 

'Контрольные вопросы и задания . . . . . . . . . . . . . . . . . . . . . . . . . . . 29\nГлава 3. Основные операции с таблицами 31\nКонтрольные вопросы и задания . . . . . . . . . . . . . . . . . . . . . . . . . . . 48\nГлава 4. Типы данных СУБД PostgreSQL 51\n4.1. Числовые типы . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 51\n4.2. Символьные (строковые) типы . . . . . . . . . . . . . . . . . . . . . . . . . 54\n4.3. Типы «дата/время» . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 56\n4.4. Логический тип . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 63\n4.5. Массивы . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 64\n4.6. Типы JSON . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 68\nКонтрольные вопросы и задания . . . . . . . . . . . . . . . . . . . . . . . . . . . 73\nГлава 5. Основы языка определения данных 95\\nОглавление\nПредисловие автора 5\nВведение 7\nГлава 1. Вве

In [32]:
# Что выдала модель со всеми данными
answer

'Базы данных — это системы хранения и управления данными, которые позволяют организовать, структурировать и безопасно хранить данные. Они необходимы для эффективного хранения и использования больших объемов данных в различных приложениях.\n\nБазы данных используются во многих областях, таких как бизнес, наука, финансы, здравоохранение и т.д., для хранения и анализа данных. Они обеспечивают возможность быстро и эффективно извлекать данные, обновлять их и управлять доступом к ним.\n\nБазы данных могут быть организованы по-разному, но наиболее распространенным типом является реляционная база данных, которая использует концепцию таблиц с связями между ними.'